# Skills examples 

This notebook is intended to highlight the different capabilities and skills of our models. This is not an exhaustive list, and the examples are meant to be simple so it's easy to understand what's going on. Note we are mostly using Lite across the examples, because of this.

We will not focus on the details around how to invoke the models and what is supported, for that please refer to the Quick Start notebook.

---

## Setup

Please make sure you followed the steps outlines in the Quick Start first. Namely, that you have the model enabled on your account and region

**Note**: _Below Examples are Text Understanding use cases and can be used with Micro, Lite or Pro models. You can try other models changing the `model_id`_

In [ ]:
import boto3
import json

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"

DEFAULT_MODEL_ID = LITE_MODEL_ID

client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
)


def invoke(prompt, model_id=DEFAULT_MODEL_ID):
    messages = [
        {"role": "user", "content": [{"text": prompt}]},
    ]

    inference_config = {
        "max_new_tokens": 600,
        "top_p": 0.9,
        "top_k": 20,
        "temperature": 0.7,
    }

    request = {
        "messages": messages,
        "inferenceConfig": inference_config,
    }

    response = client.invoke_model(modelId=model_id, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response["output"]["message"]["content"][0]["text"]

---

## Examples of skills

### Text Summarization

In this example, we download the full book Alice in Wonderland by Lewis Carroll, and ask the model to summarize it.

In [ ]:
import urllib.request


def download_url(url):
    try:
        with urllib.request.urlopen(url) as response:
            content = response.read().decode("utf-8")
            return content
    except urllib.error.URLError as e:
        print(f"Error downloading URL: {e.reason}")
        return None


book_content = download_url("https://www.gutenberg.org/files/11/11-0.txt")
print(f"{book_content[:100]=}")
print(f"{len(book_content)=}")
print(invoke(f"Summarize the following book in less than 500 words: {book_content}"))

### Content Generation

This example shows how the model can generate creative text

In [ ]:
print(invoke(f"Write an ode to the commuting to work"))

### Question Answering

This example show how the model can respond to questions about in-context content,. 

In [ ]:
manual = """
From today, washing will be easier than ever before. The Amica washing machine combines
exceptional ease of use with excellent effectiveness. Once you have read the instructions,
operating the washing machine will not be a problem.
Before being packed and leaving the factory, the safety and functions of this washing machine
were carefully tested. These tests may have resulted in some damp or traces of water being
left inside the machine.
We ask you to read the User Manual carefully before switching on the appliance. Following
the directions in this manual will protect you from any misuse.
Keep this User Manual and store it near at hand. The instructions should be followed carefully
to avoid any unfortunate accidents.
Important!
This washing machine is intended for purely domestic use to wash textiles and clothes
which are designed to be machine washed in a detergent bath.
Before plugging the washing machine into the mains, remove the safety packaging and
ensure the machine is level.
The manufacturer reserves the right to introduce changes which do not affect the operation
of the appliance.

Use
* The machine should only be used after the instructions below have been read carefully.
* Never use the machine outdoors or anywhere the temperature may fall below freezing.
* The machine should not be operated by children or anyone who has not read the instruc-
tions.

Important! Temperatures below 0°C can damage the washing machine! In the event of the
machine being stored or transported in sub-zero temperatures, it should be left to acclimatise
in a temperature above zero for 8 hours before being used.

Manufacturer’s Declaration
The manufacturer hereby declares that this product meets the requirements of the follo-
wing European directives:
* Low Voltage Directive 2014/35/UE,
* Electromagnetic Compatibility (EMC) Directive 2004/30/UE,
* ErP Directive 2009/125/EC,
* RoHS Directive 2011/65/UE,
and has thus been marked with the symbol and been issued with a declaration of com-
pliance made available to market regulators.

NOTES ON SAFETY

Use only those detergents and additives which
are designed for automatic washing machines,
especially when using the COTTON
programme. Do not use any agents containing
solvents, as these may damage certain parts of
the machine, or cause them to emit poisonous
gases. There may also be a risk of ignition and
explosion.
Ensure no unwanted objects are put inside the
appliance. Completely empty all pockets in the
clothing to be washed and fasten buttons and
zips. Place small, loose objects in a washing
bag, or inside a pillow case, as they may be
destroyed or else damage the drum or internal
tank. Do not exceed the recommended maxi-
mum load of washing.
Protect the control panel from water spillage.
Do not put washed clothes on top of the ma-
chine!
Unplug the machine before preparing the
machine for transport. Do not transport the
washing machine if drum locking bolts are not
in place. See the chapter INSTALLING THE
MACHINE. Keep the appliance in upright po-
sition when transported.
Data plate indicating basic information about
the washing machine is located at the top part
of the rear casing.
The door glass of your washing machine is hot
during operation. Do not allow children in the
vicinity of the appliance.
"""

print(
    invoke(f"""
**Question:**
Can what are the operating temperatures?
DO NOT USE INFORMATION THAT IS NOT IN REFERENCE TEXTS!

**Reference Text:**
{manual}
"""
    )
)


### Text Translation

Below we have a passage from Le Petit Prince by Antoine de Saint-Exupéry. We will ask model to identify the language, ask it (in english) to identify the book, ask it about the narrative, and finally ask it to translate to english.

In [ ]:
text = "J'ai sauté sur mes pieds comme si j'avais été frappé par la foudre. J'ai bien frotté mes yeux. J'ai bien regardé. Et j'ai vu un petit bonhomme tout à fait extraordinaire qui me considérait gravement. Voilà le meilleur portrait que, plus tard, j'ai réussi à faire de lui. Mais mon dessin, bien sûr, est beaucoup moins ravissant que le modèle. Ce n'est pas ma faute. J'avais été découragé dans ma carrière de peintre par les grandes personnes, à l'age de six ans, et je n'avais rien appris à dessiner, sauf les boas fermés et les boas ouverts. "
print(invoke(f"What language is this passage in: {text}"))
print(invoke(f"What book contains this passage: {text}"))
print(invoke(f"What did the character in this passage saw: {text}"))
print(invoke(f"Transate to english: {text}"))

### Text Analysis & Recommendations

In this example we show model's capability to understand text and sentiment

In [ ]:
review = """First off, the price alone probably led you here to start inquiring about reviews of quality. Fear not, you get a lot more bag than the price suggests. Right out of the box, soft leather without much of that leather smell. Medium brown color is classic and goes with plenty of styles. Overall, well made and provides a lot of space for storage on your travels.

I've noticed that the leather scent starts to increase with wear, which is quite pleasant. The feel remains soft and durability seems to be quite good.

Overall, great bag for the price considering you are getting a leather bag, from a known solid brand name, all for under $100.00.

The single only negative I have found with this bag is the latches used to open and close the bag. They look nice and do their job, however they are a bit cheap and aren't exactly quick and easy to use. They click into place just fine, but opening them is a fumbling challenge as they do not have much give in the spring used to release the clip. I've gotten use to it, but don't expect to quickly grab a business card at a moments notice as you'll be standing there till tomorrow fumbling with the darn clip. A bit of an exaggeration that will change from user to user, but nonetheless something worth mentioning.

Don't over think this purchase. If you have the budget for a $500.00 bag, then by all means keep searching as I do believe in getting what you pay for. However, the manufacture retail price is $300, so you won't be scraping the bottom of the barrel. Grab this one while you can!"""
print(
    invoke(
        f"Answer with positive or negative, based on the sentiment on this review: {review}"
    )
)
print(invoke(f"List the things liked in this product review: {review}"))

### Dialogue and Conversation

The following example shows how to use messages to create a back and forth chat.

In [ ]:
multi_turn_messages = []


def turn(prompt):
    multi_turn_messages.append({"role": "user", "content": [{"text": prompt}]})

    inference_config = {
        "max_new_tokens": 300,
        "top_p": 0.9,
        "top_k": 20,
        "temperature": 0.7,
    }

    request = {
        "messages": multi_turn_messages,
        "system": [{ "text": "Respond with short, concise answers" }],
        "inferenceConfig": inference_config,
    }

    response = client.invoke_model(modelId=DEFAULT_MODEL_ID, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    output_message = model_response["output"]["message"]
    multi_turn_messages.append(output_message)
    assistant_resp = output_message["content"][0]["text"]
    print(f"User: {prompt}")
    print(f"Assistant: {assistant_resp}")
    print("")


turn("My name is Alice, I'm 30 years old, I moved to Seattle 9 years ago")
turn("What's my name?")
turn("Where do I live?")
turn("At what age did I move to Seattle?")

### Structured Output

This example shows how the model can generate structured output adhering to defined schemas

In [ ]:
print(
    invoke("""
Write for each month name, the number of days in the that month and the holidays if any.
           
You MUST answer in JSON format only. Please follow the output schema below.
*Output Schema:*
[{
(month_name): {
    "number_of_days": (total number of days),
    "holidays": [( name each federal holiday )]
}
}]
"""
    )
)

### Code Explanation & Generation

In the following example ask the model to generate code. This code is very simple, but for more complex code using Nova pro is advised.

In [ ]:
print(
    invoke(
        "Write a 'odd_even' function in Python that takes in a list of numbers, and return a tuple with (odd numbers, even numbers)",
    )
)

### Classification

The following example shows how to use multimodal few-shot capabilities with multi-turns for classification.

In [ ]:
from pathlib import Path

messages = [
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/cat.jpeg').read_bytes()}}}]},
    {"role": "assistant", "content": [{"text": "fail" }]},
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/dog.jpeg').read_bytes()}}}]},
    {"role": "assistant", "content": [{"text": "pass" }]},
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/dog2.jpeg').read_bytes()}}}]},
    {"role": "assistant", "content": [{"text": "pass" }]},
    {"role": "user", "content": [{"image": {"format": "jpeg", "source": {"bytes": Path('media/cat2.jpeg').read_bytes()}}}]},
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

response = client.converse(modelId=DEFAULT_MODEL_ID, messages=messages, inferenceConfig=inf_params)

response["output"]["message"]["content"][0]["text"]